In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data, DataLoader
import string
import random

In [2]:
NUM_RAILS = 3
ALPHABET = string.ascii_uppercase
ALPHABET_SIZE = len(ALPHABET)
CHAR_TO_IDX = {char: idx for idx, char in enumerate(ALPHABET)}
IDX_TO_CHAR = {idx: char for idx, char in enumerate(ALPHABET)}

In [3]:
def railfence_cipher(text, num_rails=NUM_RAILS):
    # Prepare an empty matrix
    rails = ['' for _ in range(num_rails)]
    direction = 1  # 1 for down, -1 for up
    row = 0

    # Populate the rails by placing characters in a zigzag pattern
    for char in text.upper():
        rails[row] += char
        row += direction

        if row == 0 or row == num_rails - 1:
            direction *= -1  # Change direction when reaching the top or bottom rail

    # Join all rails into a single ciphertext
    cipher_text = ''.join(rails)
    return cipher_text


In [4]:
def generate_data(num_samples=1000, max_length=10):
    data_list = []
    for _ in range(num_samples):
        length = random.randint(1, max_length)
        plaintext = ''.join(random.choices(ALPHABET, k=length))
        ciphertext = railfence_cipher(plaintext)

        # Create node features (one-hot encoding)
        x = []
        for char in plaintext:
            one_hot = [0] * ALPHABET_SIZE
            one_hot[CHAR_TO_IDX[char]] = 1
            x.append(one_hot)
        x = torch.tensor(x, dtype=torch.float)

        # Edge index: connect each character to the next
        edge_index = []
        for i in range(len(plaintext) - 1):
            edge_index.append([i, i + 1])
            edge_index.append([i + 1, i])  # Assuming undirected edges
        if edge_index:
            edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
        else:
            edge_index = torch.empty((2, 0), dtype=torch.long)

        # Labels: target shift for each node
        y = torch.tensor([CHAR_TO_IDX[char] for char in ciphertext], dtype=torch.long)

        data = Data(x=x, edge_index=edge_index, y=y)
        data_list.append(data)
    return data_list

In [5]:
dataset = generate_data(num_samples=2000, max_length=10)
train_size = int(0.8 * len(dataset))
train_dataset = dataset[:train_size]
test_dataset = dataset[train_size:]

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

C:\Users\ekru\AppData\Roaming\Python\Python312\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [6]:
class CipherGNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(CipherGNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.lin = nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.lin(x)
        return x


In [7]:
INPUT_DIM = ALPHABET_SIZE
HIDDEN_DIM = 64
OUTPUT_DIM = ALPHABET_SIZE
EPOCHS = 50
LEARNING_RATE = 0.01

In [8]:
model = CipherGNN(INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [9]:
for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        out = model(batch)
        # Reshape outputs and labels for loss computation
        out = out.view(-1, OUTPUT_DIM)
        y = batch.y.view(-1)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * batch.num_graphs
    avg_loss = total_loss / len(train_loader.dataset)
    print(f"Epoch {epoch}, Loss: {avg_loss:.4f}")

Epoch 1, Loss: 2.6866
Epoch 2, Loss: 2.2461
Epoch 3, Loss: 2.2051
Epoch 4, Loss: 2.1838
Epoch 5, Loss: 2.1716
Epoch 6, Loss: 2.1528
Epoch 7, Loss: 2.1418
Epoch 8, Loss: 2.1282
Epoch 9, Loss: 2.1191
Epoch 10, Loss: 2.1101
Epoch 11, Loss: 2.0915
Epoch 12, Loss: 2.0865
Epoch 13, Loss: 2.0719
Epoch 14, Loss: 2.0604
Epoch 15, Loss: 2.0488
Epoch 16, Loss: 2.0444
Epoch 17, Loss: 2.0308
Epoch 18, Loss: 2.0224
Epoch 19, Loss: 2.0104
Epoch 20, Loss: 1.9998
Epoch 21, Loss: 1.9947
Epoch 22, Loss: 1.9829
Epoch 23, Loss: 1.9718
Epoch 24, Loss: 1.9684
Epoch 25, Loss: 1.9551
Epoch 26, Loss: 1.9458
Epoch 27, Loss: 1.9423
Epoch 28, Loss: 1.9290
Epoch 29, Loss: 1.9238
Epoch 30, Loss: 1.9162
Epoch 31, Loss: 1.9139
Epoch 32, Loss: 1.9099
Epoch 33, Loss: 1.8860
Epoch 34, Loss: 1.8842
Epoch 35, Loss: 1.8766
Epoch 36, Loss: 1.8753
Epoch 37, Loss: 1.8642
Epoch 38, Loss: 1.8574
Epoch 39, Loss: 1.8536
Epoch 40, Loss: 1.8392
Epoch 41, Loss: 1.8347
Epoch 42, Loss: 1.8278
Epoch 43, Loss: 1.8194
Epoch 44, Loss: 1.82

In [10]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch in test_loader:
        out = model(batch)
        preds = out.argmax(dim=1)
        correct += (preds == batch.y).sum().item()
        total += batch.y.size(0)

accuracy = correct / total
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Test Accuracy: 32.90%


In [11]:
def encrypt_with_model(model, plaintext):
    model.eval()
    # Prepare node features
    x = []
    for char in plaintext.upper():
        one_hot = [0] * ALPHABET_SIZE
        if char in CHAR_TO_IDX:
            one_hot[CHAR_TO_IDX[char]] = 1
        x.append(one_hot)
    x = torch.tensor(x, dtype=torch.float).unsqueeze(0)  # Batch size 1

    # Create edge index
    edge_index = []
    for i in range(len(plaintext) - 1):
        edge_index.append([i, i + 1])
        edge_index.append([i + 1, i])
    if edge_index:
        edge_index = torch.tensor(edge_index, dtype=torch.long).t().unsqueeze(0)
    else:
        edge_index = torch.empty((1, 2, 0), dtype=torch.long)

    # Create batch
    data = Data(x=x.squeeze(0), edge_index=edge_index.squeeze(0))
    out = model(data)
    preds = out.argmax(dim=1)
    ciphertext = ''.join([IDX_TO_CHAR[p.item()] for p in preds])
    return ciphertext

In [12]:
sample_plaintext = "KKMFURIK"
predicted_ciphertext = encrypt_with_model(model, sample_plaintext)
print(f"Plaintext: {sample_plaintext}")
print(f"Predicted Ciphertext: {predicted_ciphertext}")
print(f"Actual Ciphertext: {railfence_cipher(sample_plaintext)}")


Plaintext: KKMFURIK
Predicted Ciphertext: KKKFRRII
Actual Ciphertext: KUKFRKMI


In [13]:
def percentage_matching(str1, str2):
    if len(str1) != len(str2):
        raise ValueError("Strings must be of the same length")

    matches = sum(1 for a, b in zip(str1, str2) if a == b)
    percentage = (matches / len(str1)) * 100
    return percentage

In [14]:
percentage_matching(predicted_ciphertext,railfence_cipher(sample_plaintext))

62.5